# Common data and fun

In [ ]:
using CSV, DataFrames, Plots, StatsPlots, Contour,
Dates, GLM, Pipe, Underscores,
Statistics, StatsBase, RollingFunctions, Optim;
using Plots.PlotMeasures;
######### set global values
date_of_first_obs = Date("2020-05-01"); # cut-off date
windowsize = 7; # window size for aggregating data into windowsize-ly data
sub_sample_size = 90;  # sub-sample study size 
Nstar = 5800000; # N*, to get adjusted cases numbers
nstar = log(Nstar);
# common initial values
ω0 = 0;
ϕ0 = 1;
α0 = 0.05;
ϕ_range = collect(0.5 : 0.01 : 1);
α_range = collect(0 : 0.001 : 0.5);
ϕϕ_range = repeat(ϕ_range, inner=[size(α_range,1)]); # for vectorize
αα_range = repeat(α_range, outer=[size(ϕ_range,1)]);
Holidays2020 = [Date(2020,04,09); # through April 13
            Date(2020,05,08); Date(2020,05,21);
            Date(2020,06,01); Date(2020,06,05);
            Date(2020,12,24); Date(2020,12,25); Date(2020,12,26)];
Holidays2021 = [Date(2021,01,01); 
            Date(2021,04,01); # through April 5
            Date(2021,04,30); Date(2021,05,13);
            Date(2021,05,23); Date(2021,05,24);
            Date(2021,06,05);
            Date(2021,12,24); Date(2021,12,25); Date(2021,12,26)];
Holidays2022 = [Date(2022,01,01)];
AlphaVariant = Date(2020,12,18);
DeltaVariant = Date(2021,04,04);
OmicronVariant = Date(2021,11,24);
Variants = [AlphaVariant, DeltaVariant, OmicronVariant];


In [ ]:
######### import and clean data, 
### notes: groupby, combine, unstack, xticks labels
# Test_pos_over_time.csv
# pos.csv, the old data set
national = @pipe CSV.File("Test_pos_over_time.csv", 
                        footerskip=2, 
                        select = [1, 2, 6], 
                        typemap = Dict(Float64 => String),
                        dateformat= "yyyy-mm-dd") |> 
        DataFrame |> 
        transform!(_, :Date, 
                  :Tested => (x -> parse.(Int, replace.(x, "." => ""))) => :Nt, 
                  :NewPositive => (x -> parse.(Int, replace.(x, "." => ""))) => :Ct) |>
        transform(_, :Nt => (x -> log.(x)) => :nt,
                    :Ct => (x -> log.(x)) => :ct,
                    [:Nt, :Ct] => ((x, y) -> y ./ x) => :PosRate) |>
        transform(_, :nt => (x -> replace!(x, -Inf => 0)) => :nt,
                     :ct => (x -> replace!(x, -Inf => 0)) => :ct,
                     :PosRate => (x -> replace!(x, NaN => 0)) => :PosRate) |>
        transform(_, :nt => (x -> x - lag(x)) => :Dnt,
                     :ct => (x -> x - lag(x)) => :Dct,
                     :Date => (x -> Dates.month.(x)) => :Month,
                     :Date => (x -> Dates.year.(x)) => :Year,
                     :Date => (x -> Dates.dayofweek.(x)) => :Day) |>
        dropmissing(_) |>
        transform(_, :Day => (x -> ifelse.(x .== 1, 1, 0)) => :Mon,
                     :Day => (x -> ifelse.(x .== 2, 1, 0)) => :Tues,
                     :Day => (x -> ifelse.(x .== 3, 1, 0)) => :Wed,
                     :Day => (x -> ifelse.(x .== 4, 1, 0)) => :Thurs,
                     :Day => (x -> ifelse.(x .== 5, 1, 0)) => :Fri,
                     :Day => (x -> ifelse.(x .== 6, 1, 0)) => :Sat,
                     :Day => (x -> ifelse.(x .== 7, 1, 0)) => :Sun)

national_daily = @pipe national |>
                transform(_, :nt => (x -> lag(x)) => :nt_lag) |> 
                subset(_, :Date => ByRow(D -> D >= date_of_first_obs)); # drop data before 2020-07-01 ;

national_weekly = @pipe national |>
        select(_, :Date,
                  :Month,
                  :Year,
                  :Day,
                  :Nt => (v -> runmean(v, windowsize) * windowsize) => :Nt, 
                  :Ct => (v -> runmean(v, windowsize) * windowsize) => :Ct)  |>
        delete!(_, 1: (windowsize - 1)) |>
        transform(_, :Nt => (x -> log.(x)) => :nt,
                     :Ct => (x -> log.(x)) => :ct,
                     [:Nt, :Ct] =>  ((x, y) -> y ./ x) => :PosRate) |>
        transform(_, :nt => (x -> replace!(x, -Inf => 0)) => :nt,
                     :ct => (x -> replace!(x, -Inf => 0)) => :ct,
                     :PosRate => (x -> replace!(x, NaN => 0)) => :PosRate) |>
        transform(_, :nt => (x -> x - lag(x)) => :Dnt,
                     :ct => (x -> x - lag(x)) => :Dct,
                     :nt => (x -> lag(x)) => :nt_lag) |>
        dropmissing(_) |>
        subset(_, :Date => ByRow(D -> D >= date_of_first_obs)); # drop data before 2020-07-01 ;

national_7dd = @pipe national |>
        select(_, :Date,
                  :Month,
                  :Year,
                  :Day,
                  :Nt,
                  :Ct,
                  :nt,
                  :ct,
                  :nt => (x -> x - lag(x, 7)) => :Dnt,
                  :ct => (x -> x - lag(x, 7)) => :Dct,
                  :nt => (x -> lag(x, 7)) => :nt_lag) |>
        dropmissing(_) |>
        subset(_, :Date => ByRow(D -> D >= date_of_first_obs)); # drop data before 2020-07-01 ;

# Common functions 
function OLS_produce_initial(data)
        OLS0 = lm(@formula(Dct ~ Dnt), data);
        DctHat = predict(OLS0);
        σ0 = std(data.Dct - DctHat); # use OLS results as initial values for sigma and theta0
        βt0 = coef(OLS0)[2];
        θt0 = log(βt0 / (1 - βt0));
        return(σ0, θt0)
end


In [ ]:
#CSV.write("national.csv", national)
function season_Dnt(data, which_year)
    @pipe data |> filter(:Year => x -> x == which_year, _) |>
    groupby(_, [:Month, :Day]) |>
    combine(_, :Dnt => mean => :bar) |>
    sort!(_, [:Day, :Month]) |>
    @df _ plot(:Day, :bar, 
    group = :Month,
    xticks = (1:7, Dates.dayname.(1:7)),
    label = :none,
    ylab = "nt - its weekly mean",
    xrotation = 45,
    title = which_year);
end

function season_Dct(data, which_year)
    @pipe data |> filter(:Year => x -> x == which_year, _) |>
    groupby(_, [:Month, :Day]) |>
    combine(_, :Dct => mean => :bar) |>
    sort!(_, [:Day, :Month]) |>
    @df _ plot(:Day, :bar, 
    group = :Month,
    xticks = (1:7, Dates.dayname.(1:7)),
    label = :none,
    ylab = "ct - its weekly mean",
    xrotation = 45,
    title = which_year);
end

# Data Plots

In [ ]:
first_Mon_index = findfirst(x -> x .== 1, national.Mon)
last_Sun_index = findlast(x -> x .== 1, national.Sun)
seasonal_national_data = @pipe national[first_Mon_index : last_Sun_index, :] |>
    select(_, Not([:Dnt, :Dct])) |>
    transform(_, :nt => (x -> x .- repeat(rollmean(x, 7)[1:7:end], inner = 7)) => :Dnt,
                 :ct => (x -> x .- repeat(rollmean(x, 7)[1:7:end], inner = 7)) => :Dct);

In [ ]:
 function important_dates_plot(data, variable, data_type)
    x = data.Date
    if variable == "Dnt"
        y = data.Dnt
        cc = "blue"
    elseif variable == "Dct"
        y = data.Dct
        cc = "red"
    end   
    p = plot(x, y, ylabel = variable,
    label = :none, color = cc, leg = :bottomleft, size = (1100, 300),
    xticks = collect(data.Date[1]:Month(3):data.Date[nrow(data)]),
    left_margin = 50px,
    title = data_type);
    vline!(Holidays2020, color = :green, label = "Holidays", alpha = 0.5);
    vline!(Holidays2021, color = :green, label = :none, alpha = 0.5);
    vline!(Holidays2022, color = :green, label = :none, alpha = 0.5);
    vspan!([AlphaVariant, DeltaVariant], label = :none, color = :red, alpha = 0.1);
    vspan!([DeltaVariant, OmicronVariant], label = :none, color = :yellow, alpha = 0.1);
    vspan!([OmicronVariant, data.Date[nrow(data)]], label = :none, color = :green, alpha = 0.1);
    return(p)
end

In [ ]:
ImpDate_Daily1 = important_dates_plot(national, "Dnt", "Daily");
ImpDate_Daily2 = important_dates_plot(national, "Dct", "Daily");
ImpDate_Weekly1 = important_dates_plot(national_weekly, "Dnt", "Weekly");
ImpDate_Weekly2 = important_dates_plot(national_weekly, "Dct", "Weekly");
ImpDate_7dd1 = important_dates_plot(national_7dd, "Dnt", "7DD");
ImpDate_7dd2 = important_dates_plot(national_7dd, "Dct", "7DD");

In [ ]:

plot_ntct_daily = @df national_daily plot(:Date, :Nt, yaxis = :log, ylabel = "Nt values",
        label = "log(Nt)", color = "blue", leg = :topleft)
        @df national_daily plot!(twinx(), :Date, :Ct, yaxis = :log, ylabel = "Ct values",
        label = "log(Ct)", color = "red", leg = :bottomright);
    
plot_ntct_weekly = @df national_weekly plot(:Date, :Nt, yaxis = :log, ylabel = "Nt values",
        label = "log(Nt)", color = "blue", leg = :topleft)
         @df national_weekly plot!(twinx(), :Date, :Ct, yaxis = :log, ylabel = "Ct values",
        label = "log(Ct)", color = "red", leg = :bottomright);

In [ ]:
plot_Dnt_daily = @df national_daily plot(:Date, :Dnt, ylabel = "Δnt",
        label = :none, color = "blue", leg = :bottomleft);
plot_Dct_daily = @df national_daily plot(:Date, :Dct, ylabel = "Δct",
        label = :none, color = "red", leg = :bottomright);
plot_Dnt_weekly = @df national_weekly plot(:Date, :Dnt, ylabel = "Δnt",
        label = :none, color = "blue", leg = :bottomleft);
plot_Dct_weekly = @df national_weekly plot(:Date, :Dct, ylabel = "Δct",
        label = :none, color = "red", leg = :bottomright);
plot_Dnt_7dd = @df national_7dd plot(:Date, :Dnt, ylabel = "Δnt",
        label = :none, color = "blue", leg = :bottomleft);
plot_Dct_7dd = @df national_7dd plot(:Date, :Dct, ylabel = "Δct",
        label = :none, color = "red", leg = :bottomright);

In [ ]:
l = @layout [a; b; grid(2,2){0.5h}];
n2020 = season_Dnt(seasonal_national_data, 2020);
c2020 = season_Dct(seasonal_national_data, 2020);
n2021 = season_Dnt(seasonal_national_data, 2021);
c2021 = season_Dct(seasonal_national_data, 2021);
plot(ImpDate_Daily1, 
    ImpDate_Daily2,   
    n2020, n2021,
    c2020, c2021, 
    layout = l,
    size = (800,900))
#png("1-1-daily")

In [ ]:
l = @layout [a; b; grid(2,2){0.5h}];
n2020 = season_Dnt(national_weekly, 2020);
c2020 = season_Dct(national_weekly, 2020);
n2021 = season_Dnt(national_weekly, 2021);
c2021 = season_Dct(national_weekly, 2021);
plot(ImpDate_Weekly1, 
    ImpDate_Weekly2,  
    n2020, n2021,
    c2020, c2021, 
    layout = l,
    size = (800,900))
#png("1-2-weekly")

In [ ]:
l = @layout [a; b; grid(2,2){0.5h}];
n2020 = season_Dnt(national_7dd, 2020);
c2020 = season_Dct(national_7dd, 2020);
n2021 = season_Dnt(national_7dd, 2021);
c2021 = season_Dct(national_7dd, 2021);
plot(ImpDate_7dd1, 
    ImpDate_7dd2,   
    n2020, n2021,
    c2020, c2021, 
    layout = l,
    size = (800,900))
#png("1-3-7dd")

# Model 1

## logl and recoverTheta

In [ ]:
βasθ(θ) = 1 / (1 + exp(-θ));

function recoverTheta1(par, data)
    x = data.Dnt;
    y = data.Dct;
    T = nrow(data);
    ω = par[1]; 
    ϕ = par[2];
    α = par[3]; 
    σ = par[4];
    θ0 = par[5];
    θ = [];
    push!(θ, θ0)
    for i in 1:(T-1)
        v = ω + ϕ * θ[i] + α * (1 / σ) * sign(x[i]) * ( y[i] - x[i] * βasθ(θ[i]) - par[6] * data.Mon[i]
        - par[7] * data.Tues[i]
        - par[8] * data.Wed[i]
        - par[9] * data.Thurs[i]
        - par[10] * data.Fri[i]
        - par[11] * data.Sat[i]
        - par[12] * data.Sun[i]) 
        push!(θ, v)
    end
    return θ
end

function Ut1(par, data)  
    x = data.Dnt;
    y = data.Dct;
    θ = recoverTheta1(par, data);
    # par[6] = Mon, ... par[12] = Sun
    Ut = y .- βasθ.(θ) .* x 
        .- par[6] .* data.Mon 
        .- par[7] .* data.Tues
        .- par[8] .* data.Wed
        .- par[9] .* data.Thurs
        .- par[10] .* data.Fri
        .- par[11] .* data.Sat
        .- par[12] .* data.Sun
    return(Ut)
end

function logL1(par, data)
    T = nrow(data);
    σ = par[4];
    Ut = Ut1(par, data)
    ll = - T * log(σ^2) / 2 - sum(Ut .^ 2)/(2 * σ^2)  
    return(ll)
end


## α, ϕ and logL, Optim

In [35]:
# new optimize function
# use a set of different fixed values for α, then use optim, compare ll
# chose the α with highest ll and all other parameters
function manual_restricted_alpha(σ_ini, θ0_ini, w1_ini, w2_ini, 
                                w3_ini, w4_ini, w5_ini, w6_ini, w7_ini, 
                                α_range, data)
    ll = []
    all_other_para = []
    for α in α_range
        fix_α_fun(σ, θ0, w1, w2, w3, w4, w5, w6, w7) = 
            logL1([0, 1, α, σ, θ0, w1, w2, w3, w4, w5, w6, w7], data)
        obj = para -> -fix_α_fun(para[1], para[2], para[3], para[4], para[5], 
                                para[6], para[7], para[8], para[9])
        res = optimize(obj, [σ_ini, θ0_ini, w1_ini, w2_ini, 
                            w3_ini, w4_ini, w5_ini, w6_ini, w7_ini]) 
        new_ll = -Optim.minimum(res)
        push!(ll, new_ll)
        if new_ll >= max(ll ...)
            all_other_para = copy(Optim.minimizer(res))
        end  
    end
    return(ll, all_other_para)
end

manual_restricted_alpha (generic function with 1 method)

In [36]:
manual_restricted_alpha(0.2, 0, 
                        1, 1, 1, 1, -1, -1, -1, 
                                α_range, national_daily)

(Any[896.227419742002, 930.3138486175017, 930.2043938502742, 930.0951177049111, 929.9645263949567, 929.8806127674982, 929.7732888613558, 929.6669671265073, 929.563597283544, 929.4594081879598  …  876.433399888773, 876.4261200902624, 876.4174256602587, 872.5198159884794, 876.4003497253037, 876.3881312950462, 872.5008927609013, 876.371979783439, 876.3644427601645, 876.3534065435191], [0.16602352753748398, 2.8906142414922162, 114.28510970165098, -28.436864821894133, 6.6434399416568946, -2.413643938962036, 48.67764691790171, 73.04107320378647, -19.49595656309859])

In [ ]:
## By optimized initial values
function find_σ_θ_ini_1(ω0, ϕ0, α0, data)
    findIni(a, b, w1, w2, w3, w4, w5, w6, w7) = 
        logL1([ω0, ϕ0, α0, a, b, w1, w2, w3, w4, w5, w6, w7], data); 
    # a = σ; b = θ0, w1 = Mon, ... w7 = Sun; define a function in sigma and theta0
    obj0 = para -> -findIni(para[1], para[2], 
                            para[3], para[4], para[5], para[6], para[7], para[8], para[9])
    σ0, θt0 = OLS_produce_initial(data)
    Optim0 = optimize(obj0, [σ0, θt0, 0, 0, 0, 0, 0, 0, 0]) # optimize sigma and theta0 based on input values of omega, phi, sigma
    new_initial = Optim.minimizer(Optim0);
    σ = new_initial[1];
    θ0 = new_initial[2];
    w1 = new_initial[3];
    w2 = new_initial[4];
    w3 = new_initial[5];
    w4 = new_initial[6];
    w5 = new_initial[7];
    w6 = new_initial[8];
    w7 = new_initial[9];
    return(σ, θ0, w1, w2, w3, w4, w5, w6, w7)
end

function manual_optim_αOnly_1(ω0, ϕ0, α0, α_range, data)
    σ, θ0, w1, w2, w3, w4, w5, w6, w7 = find_σ_θ_ini_1(ω0, ϕ0, α0, data);
    f(α) = logL1([ω0, ϕ0, α, σ, θ0, w1, w2, w3, w4, w5, w6, w7], data);
    ll_range = f.(α_range);
    max_ll, max_index = findmax(ll_range);
    max_α = α_range[max_index];
    Optimized_Parameters = [ω0, ϕ0, max_α, σ, θ0, w1, w2, w3, w4, w5, w6, w7];
    return(Optimized_Parameters, max_ll, f)
end

function manual_optim_α_ϕ_1(ω0, ϕ0, α0, ϕϕ_range, αα_range, data)
    σ, θ0 = find_σ_θ_ini_1(ω0, ϕ0, α0, data);
    f(ϕ, α) = logL1([ω0, ϕ, α, σ, θ0], data);
    ll_range = f.(ϕϕ_range, αα_range);
    max_ll, max_index = findmax(ll_range);
    max_ϕ = ϕϕ_range[max_index];
    max_α = αα_range[max_index];
    Optimized_Parameters = [ω0, max_ϕ, max_α, σ, θ0];
    return(Optimized_Parameters, max_ll, f)
end

function Plots2D_α_1(optim_results, data)
    Optimized_Parameters, max_ll, f = optim_results;
    ω, ϕ, α, σ, θ0 = Optimized_Parameters;
    Uhat = Ut1([ω, ϕ, α, σ, θ0], data);
    θ = recoverTheta1([ω, ϕ, α, σ, θ0], data);
    β = βasθ.(θ);
    α_plot = plot(f, α_range,
                ylabel = "logL", 
                xlabel = "α", 
                title = string("ω = ", round(ω, digits=2), "\n",
                                "ϕ is fixed ", ϕ,  "\n",
                                "σ = ", round(σ, digits =2), "\n",
                                "β0 = ", round(βasθ(θ0), digits = 2)),
                label = :none);
    β_plot = plot(data.Date, β, 
                    ylabel = "βt", xlabel = "Date", 
                    title = string("ϕ is fixed ", ϕ, "\n",
                                "α hat = ", round(α, digits = 2), "\n",
                                "max ll = ", round(max_ll, digits = 2)),
                    label = :none,
                    ylim = (0, 1));
    U_plot = plot(data.Date, Uhat, 
                    ylabel = "Ut hat", xlabel = "Date", 
                    title = string("ϕ is fixed ", ϕ, "\n",
                                "α hat = ", round(α, digits = 2), "\n",
                                "max ll = ", round(max_ll, digits = 2)),
    label = :none);
    U2_plot = plot(data.Date, Uhat .^ 2, 
                    ylabel = "Ut square", xlabel = "Date", 
                    title = string("ϕ is fixed ", ϕ, "\n",
                                "α hat = ", round(α, digits = 2), "\n",
                                "max ll = ", round(max_ll, digits = 2)),
    label = :none);
    return(α_plot, β_plot, U_plot, U2_plot)
end

function Plots3D_α_ϕ_1(optim_results, data)
    Optimized_Parameters, max_ll, f = optim_results;
    ω, ϕ, α, σ, θ0 = Optimized_Parameters;
    Uhat = Ut1([ω, ϕ, α, σ, θ0], data);
    θ = recoverTheta1([ω, ϕ, α, σ, θ0], data);
    β = βasθ.(θ);
    α_plot = surface(ϕ_range, α_range, f,
                xlabel = "ϕ", 
                ylabel = "α",
                zlabel = "logL",
                title = string("ω = ", round(ω, digits=2), "\n",
                                "σ = ", round(σ, digits =2), "\n",
                                "β0 = ", round(βasθ(θ0), digits = 2)));
    β_plot = plot(data.Date, β, 
                    ylabel = "βt", xlabel = "Date", 
                    title = string("ϕ hat = ", round(ϕ, digits =2), "\n",
                                "α hat = ", round(α, digits = 2), "\n",
                                "max ll = ", round(max_ll, digits = 2)),
                    label = :none,
                    ylim = (0, 1));
    U_plot = plot(data.Date, Uhat, 
                    ylabel = "Ut hat", xlabel = "Date", 
                    title = string("ϕ hat = ", round(ϕ, digits =2), "\n",
                                "α hat = ", round(α, digits = 2), "\n",
                                "max ll = ", round(max_ll, digits = 2)),
                    label = :none);
    U2_plot = plot(data.Date, Uhat .^ 2, 
                    ylabel = "Ut square", xlabel = "Date", 
                    title = string("ϕ hat = ", ϕ, "\n",
                                "α hat = ", round(α, digits = 2), "\n",
                                "max ll = ", round(max_ll, digits = 2)),
    label = :none);
    return(α_plot, β_plot, U_plot, U2_plot)
end

## 1 - Run Functions

In [ ]:
# Daily Estimation
Restricted_Daily = manual_optim_αOnly_1(ω0, ϕ0, α0, α_range, national_daily); 
Unrestricted_Daily = manual_optim_α_ϕ_1(ω0, ϕ0, α0, ϕϕ_range, αα_range, national_daily);

In [ ]:
function select_alpha()
ll = []
for α in α_range
    v = manual_optim_αOnly_1(ω0, ϕ0, α0, α_range, national_daily)[2]
    push!(ll, v)
end
return(ll)
end

In [ ]:
α0 = 0.05
α_range = collect(0 : 0.001 : 0.5);
results = manual_optim_αOnly_1(ω0, ϕ0, α0, α_range, national_daily)
b = results[1]
maxll = results[2]
β = βasθ.(recoverTheta1(b, national_daily))
score_hat = Ut1(b, national_daily) .* national_daily.Dnt / b[4];

In [ ]:
plot(β, ylim = (0, 1))

In [ ]:
b

In [ ]:
density(score_hat, label = "Ut * Xt / σ", ylabel = "Density")

In [ ]:
plot(national_daily.Date, score_hat, ylabel = "Ut * Xt / σ", label = :none,
    xticks = collect(national_daily.Date[1]:Month(2):national_daily.Date[nrow(national_daily)]),
    xrotation = 45)

In [ ]:
# Daily Plots
α_only1_daily_p1, α_only1_daily_p2, α_only1_daily_pUt, α_only1_daily_pUt2 = Plots2D_α_1(
    Restricted_Daily, national_daily);
α_and_ϕ1_daily_p1, α_and_ϕ1_daily_p2, α_and_ϕ1_daily_pUt, α_and_ϕ1_daily_pUt2 = Plots3D_α_ϕ_1(
    Unrestricted_Daily, national_daily);

In [ ]:
# Weekly Estimation
Restricted_Weekly = manual_optim_αOnly_1(ω0, ϕ0, α0, α_range, national_weekly); 
Unrestricted_Weekly = manual_optim_α_ϕ_1(ω0, ϕ0, α0, ϕϕ_range, αα_range, national_weekly);

In [ ]:
# Weekly Plots
α_only1_weekly_p1, α_only1_weekly_p2, α_only1_weekly_pUt, α_only1_weekly_pUt2 = Plots2D_α_1(
    Restricted_Weekly,  national_weekly);
α_and_ϕ1_weekly_p1, α_and_ϕ1_weekly_p2, α_and_ϕ1_weekly_pUt, α_and_ϕ1_weekly_pUt2 = Plots3D_α_ϕ_1(
    Unrestricted_Weekly, national_weekly);

In [ ]:
plot(α_only1_daily_p2, α_and_ϕ1_daily_p2)

In [ ]:
# 7DD Estimation
Restricted_7DD = manual_optim_αOnly_1(ω0, ϕ0, α0, α_range, national_7dd); 
Unrestricted_7DD = manual_optim_α_ϕ_1(ω0, ϕ0, α0, ϕϕ_range, αα_range, national_7dd);

In [ ]:
# 7DD Plots
α_only1_7dd_p1, α_only1_7dd_p2, α_only1_7dd_pUt, α_only1_7dd_pUt2 = Plots2D_α_1(
    Restricted_7DD, national_7dd);
α_and_ϕ1_7dd_p1, α_and_ϕ1_7dd_p2, α_and_ϕ1_7dd_pUt, α_and_ϕ1_7dd_pUt2 = Plots3D_α_ϕ_1(
    Unrestricted_7DD, national_7dd);

# Model 2

## logl and recoverTheta

In [ ]:
function recoverTheta2(par, data)
    nt = data.nt;
    nt_lag = data.nt_lag;
    y = data.Dct;
    T = nrow(data);
    ω = par[1]; 
    ϕ = par[2];
    α = par[3]; 
    σ = par[4];
    θ0 = par[5];
    θ1 = par[6]
    θ = [];
    push!(θ, θ0);
    push!(θ, θ1);
    for i in 2:(T-1)
        v = ω + 
        ϕ * θ[i] -
        α / σ * (y[i] - 
                    nt[i] * βasθ(θ[i]) + 
                    nt_lag[i] * βasθ(θ[i-1]) + 
                    nstar * βasθ(θ[i]) -
                    nstar * βasθ(θ[i-1]))
        push!(θ, v)
    end
    return θ
end

function Ut2(par, data)
    nt = data.nt;
    nt_lag = data.nt_lag;
    y = data.Dct;
    T = nrow(data);
    θ = recoverTheta2(par, data);
    β = βasθ.(θ);
    Ut = [];
    u1 = y[1] - β[1] * (nt[1] - nt_lag[1]);  # since Ut[1] cannot be calculated
    push!(Ut, u1);
    for i in 2:T
        u = y[i] - nt[i] * β[i] + nt_lag[i] * β[i-1] + 
        nstar * β[i] - nstar * β[i-1];
        push!(Ut, u)
    end
    return(Ut)
end

function logL2(par, data)
    T = nrow(data);
    σ = par[4];
    Ut = Ut2(par, data)
    ll = - T * log(σ^2) / 2 - sum(Ut .^ 2)/(2 * σ^2)  
    return(ll)
end



## α, ϕ and logL, By optim, fast

In [ ]:
## By optimized initial values
function find_σ_θ_ini_2(ω0, ϕ0, α0, data)
    findIni(a, b, c) = logL2([ω0, ϕ0, α0, a, b, c], data); # a = σ; b = θ0; c = θ1; define a function in sigma and theta0
    obj0 = para -> -findIni(para[1], para[2], para[3])
    σ0, θt0 = OLS_produce_initial(data);
    θt1 = θt0;
    Optim0 = optimize(obj0, [σ0, θt0, θt1]); # optimize sigma and theta0 based on input values of omega, phi, sigma
    new_initial = Optim.minimizer(Optim0);
    σ = new_initial[1];
    θ0 = new_initial[2];
    θ1 = new_initial[3];
    return(σ, θ0, θ1)
end

function manual_optim_αOnly_2(ω0, ϕ0, α0, α_range, data)
    σ, θ0, θ1 = find_σ_θ_ini_2(ω0, ϕ0, α0, data);
    f(α) = logL2([ω0, ϕ0, α, σ, θ0, θ1], data);
    ll_range = f.(α_range);
    max_ll, max_index = findmax(ll_range);
    max_α = α_range[max_index];
    Optimized_Parameters = [ω0, ϕ0, max_α, σ, θ0, θ1];
    return(Optimized_Parameters, max_ll, f)
end

function manual_optim_α_ϕ_2(ω0, ϕ0, α0, ϕϕ_range, αα_range, data)
    σ, θ0, θ1 = find_σ_θ_ini_2(ω0, ϕ0, α0, data);
    f(ϕ, α) = logL2([ω0, ϕ, α, σ, θ0, θ1], data);
    
    ll_range = f.(ϕϕ_range, αα_range);
    max_ll, max_index = findmax(ll_range);
    max_ϕ = ϕϕ_range[max_index];
    max_α = αα_range[max_index];
    Optimized_Parameters = [ω0, max_ϕ, max_α, σ, θ0, θ1];
    return(Optimized_Parameters, max_ll, f)
end

function Plots2D_α_2(optim_results, data)
    Optimized_Parameters, max_ll, f = optim_results
    ω, ϕ, α, σ, θ0, θ1 = Optimized_Parameters;
    Uhat = Ut2([ω, ϕ, α, σ, θ0, θ1], data);
    θ = recoverTheta2([ω, ϕ, α, σ, θ0,  θ1], data);
    β = βasθ.(θ);
    α_plot = plot(f, α_range,
                ylabel = "logL", 
                xlabel = "α", 
                title = string("ω = ", round(ω, digits=2), "\n",
                                "ϕ is fixed ", ϕ,  "\n",
                                "σ = ", round(σ, digits =2), "\n",
                                "β0 = ", round(βasθ(θ0), digits = 2), "\n",
                                "β1 = ", round(βasθ(θ1), digits = 2)),
                label = :none);
    β_plot = plot(data.Date, β, 
                    ylabel = "βt", xlabel = "Date", 
                    title = string("ϕ is fixed ", ϕ, "\n",
                                "α hat = ", round(α, digits = 2), "\n",
                                "max ll = ", round(max_ll, digits = 2)),
                    label = :none,
                    ylim = (0, 1));
    U_plot = plot(data.Date, Uhat, 
                    ylabel = "Ut hat", xlabel = "Date", 
                    title = string("ϕ is fixed ", ϕ, "\n",
                                "α hat = ", round(α, digits = 2), "\n",
                                "max ll = ", round(max_ll, digits = 2)),
    label = :none);
    U2_plot = plot(data.Date, Uhat .^ 2, 
                    ylabel = "Ut square", xlabel = "Date", 
                    title = string("ϕ is fixed ", ϕ, "\n",
                                "α hat = ", round(α, digits = 2), "\n",
                                "max ll = ", round(max_ll, digits = 2)),
    label = :none);
    return(α_plot, β_plot, U_plot, U2_plot)
end

function Plots3D_α_ϕ_2(optim_results, data)
    Optimized_Parameters, max_ll, f = optim_results
    ω, ϕ, α, σ, θ0, θ1 = Optimized_Parameters;
    Uhat = Ut2([ω, ϕ, α, σ, θ0, θ1], data);
    θ = recoverTheta2([ω, ϕ, α, σ, θ0, θ1], data);
    β = βasθ.(θ);
    α_plot = surface(ϕ_range, α_range, f,
                xlabel = "ϕ", 
                ylabel = "α",
                zlabel = "logL",
                title = string("ω = ", round(ω, digits=2), "\n",
                                "σ = ", round(σ, digits =2), "\n",
                                "β0 = ", round(βasθ(θ0), digits = 2), "\n",
                                "β1 = ", round(βasθ(θ1), digits = 2)));
    β_plot = plot(data.Date, β, 
                    ylabel = "βt", xlabel = "Date", 
                    title = string("ϕ hat = ", round(ϕ, digits =2), "\n",
                                "α hat = ", round(α, digits = 2), "\n",
                                "max ll = ", round(max_ll, digits = 2)),
                    label = :none,
                    ylim = (0, 1));
    U_plot = plot(data.Date, Uhat, 
                    ylabel = "Ut hat", xlabel = "Date", 
                    title = string("ϕ hat = ", round(ϕ, digits =2), "\n",
                                "α hat = ", round(α, digits = 2), "\n",
                                "max ll = ", round(max_ll, digits = 2)),
                    label = :none);
    U2_plot = plot(data.Date, Uhat .^ 2, 
                    ylabel = "Ut square", xlabel = "Date", 
                    title = string("ϕ hat = ", ϕ, "\n",
                                "α hat = ", round(α, digits = 2), "\n",
                                "max ll = ", round(max_ll, digits = 2)),
    label = :none);
    return(α_plot, β_plot, U_plot, U2_plot)
end

## 2-Run Functions

In [ ]:
# Daily Estimation
Restricted_Daily_2 = manual_optim_αOnly_2(ω0, ϕ0, α0, α_range, national_daily); 
Unrestricted_Daily_2 = manual_optim_α_ϕ_2(ω0, ϕ0, α0, ϕϕ_range, αα_range, national_daily);

In [ ]:
# Daily Plots
α_only2_daily_p1, α_only2_daily_p2, α_only2_daily_pUt, α_only2_daily_pUt2 = Plots2D_α_2(
    Restricted_Daily_2, national_daily);
α_and_ϕ2_daily_p1, α_and_ϕ2_daily_p2, α_and_ϕ2_daily_pUt, α_and_ϕ2_daily_pUt2 = Plots3D_α_ϕ_2(
    Unrestricted_Daily_2, national_daily);


In [ ]:
# Weekly Estimation
Restricted_Weekly_2 = manual_optim_αOnly_2(ω0, ϕ0, α0, α_range, national_weekly); 
Unrestricted_Weekly_2 = manual_optim_α_ϕ_2(ω0, ϕ0, α0, ϕϕ_range, αα_range, national_weekly);

In [ ]:
# Weekly Plots
α_only2_weekly_p1, α_only2_weekly_p2, α_only2_weekly_pUt, α_only2_weekly_pUt2 = Plots2D_α_2(
    Restricted_Weekly_2,  national_weekly);
α_and_ϕ2_weekly_p1, α_and_ϕ2_weekly_p2, α_and_ϕ2_weekly_pUt, α_and_ϕ2_weekly_pUt2 =  Plots3D_α_ϕ_2(
    Unrestricted_Weekly_2, national_weekly);


In [ ]:
# 7DD Estimation
Restricted_7DD_2 = manual_optim_αOnly_2(ω0, ϕ0, α0, α_range, national_7dd); 
Unrestricted_7DD_2 = manual_optim_α_ϕ_2(ω0, ϕ0, α0, ϕϕ_range, αα_range, national_7dd);

In [ ]:
# 7DD Plots
α_only2_7dd_p1, α_only2_7dd_p2, α_only2_7dd_pUt, α_only2_7dd_pUt2 = Plots2D_α_2(
    Restricted_7DD_2, national_7dd);
α_and_ϕ2_7dd_p1, α_and_ϕ2_7dd_p2, α_and_ϕ2_7dd_pUt, α_and_ϕ2_7dd_pUt2 =  Plots3D_α_ϕ_2(
    Unrestricted_7DD_2, national_7dd);

# Out-of-Sample Functions

In [ ]:
function Out_Sample_Perform(data, which_model, α_only, step)
    T = nrow(data);
    in_sample_logL = [];
    out_sample_error = [];
    if which_model == 1 && α_only == true
        for i in 1:step
            ss = T - step + i - 1;
            train = data[i:ss, :];
            test = data[ss:(ss+1), :];
            Optimized_Parameters, max_ll, _ = manual_optim_αOnly_1(ω0, ϕ0, α0, α_range, train);
            ω, ϕ, α, σ =  Optimized_Parameters[1:4]
            θss_for_test = recoverTheta1(Optimized_Parameters, train)[ss - i + 1];
            oou_para = [ω, ϕ, α, σ, θss_for_test]
            oou = Ut1(oou_para, test)[1]
            push!(in_sample_logL, max_ll);
            push!(out_sample_error, oou)
        end
    elseif which_model == 1 && α_only == false
        for i in 1:step
            ss = T - step + i - 1;
            train = data[i:ss, :];
            test = data[ss:(ss+1), :];
            Optimized_Parameters, max_ll, _ = manual_optim_α_ϕ_1(ω0, ϕ0, α0, ϕϕ_range, αα_range, train);
            ω, ϕ, α, σ =  Optimized_Parameters[1:4]
            θss_for_test = recoverTheta1(Optimized_Parameters, train)[ss - i + 1];
            oou_para = [ω, ϕ, α, σ, θss_for_test]
            oou = Ut1(oou_para, test)[1]
            push!(in_sample_logL, max_ll);
            push!(out_sample_error, oou)
        end
    elseif which_model == 2 && α_only == true
        for i in 1:step
            ss = T - step + i - 1;
            train = data[i:ss, :];
            test = data[(ss-1):ss, :];
            Optimized_Parameters, max_ll, _ = manual_optim_αOnly_2(ω0, ϕ0, α0, α_range, train);
            ω, ϕ, α, σ =  Optimized_Parameters[1:4]
            θss_for_test = recoverTheta2(Optimized_Parameters, train)[(ss-i) : (ss - i +1)];
            oou_para = [ω, ϕ, α, σ, θss_for_test[1], θss_for_test[2]];
            oou = Ut2(oou_para, test)[2]
            push!(in_sample_logL, max_ll);
            push!(out_sample_error, oou)
        end
    elseif which_model ==2 && α_only == false
        for i in 1:step
            ss = T - step + i - 1;
            train = data[i:ss, :];
            test = data[(ss-1):ss, :];
            Optimized_Parameters, max_ll, _ = manual_optim_α_ϕ_2(ω0, ϕ0, α0, ϕϕ_range, αα_range, train);
            ω, ϕ, α, σ =  Optimized_Parameters[1:4]
            θss_for_test = recoverTheta2(Optimized_Parameters, train)[(ss-i) : (ss - i +1)];
            oou_para = [ω, ϕ, α, σ, θss_for_test[1], θss_for_test[2]];
            oou = Ut2(oou_para, test)[2]
            push!(in_sample_logL, max_ll);
            push!(out_sample_error, oou)
        end 
    else
        println("Wrong input")
    end
    return(in_sample_logL, out_sample_error)
    end

function compare_plots(data, table, data_type)
    T = nrow(data);
    logL_plot = plot(data.Date[T - step + 1 : T], table.logL_restricted_M1, label = "M1 restricted", title = data_type)
        plot!(data.Date[T - step + 1 : T], table.logL_restricted_M2, label = "M2 restricted")
        plot!(data.Date[T - step + 1 : T], table.logL_unrestricted_M1, label = "M1 unrestricted")
        plot!(data.Date[T - step + 1 : T], table.logL_unrestricted_M2, label = "M2 unrestricted");
    uhat_plot = plot(data.Date[T - step + 1 : T], abs.(table.uhat_restricted_M1), label = "M1 restricted", title = data_type)
        plot!(data.Date[T - step + 1 : T], abs.(table.uhat_restricted_M2), label = "M2 restricted")
        plot!(data.Date[T - step + 1 : T], abs.(table.uhat_unrestricted_M1), label = "M1 unrestricted")
        plot!(data.Date[T - step + 1 : T], abs.(table.uhat_unrestricted_M2), label = "M2 unrestricted");
    return(logL_plot, uhat_plot)
end

# this is a code chunk

step = 100; # saved results, no need to run again for now
d1t_in, d1t_out = Out_Sample_Perform(national_daily, 1, true, step);
d1f_in, d1f_out = Out_Sample_Perform(national_daily, 1, false, step);
d2t_in, d2t_out = Out_Sample_Perform(national_daily, 2, true, step);
d2f_in, d2f_out = Out_Sample_Perform(national_daily, 2, false, step);

# this is a code chunk

compare_daily = DataFrame(logL_restricted_M1 = d1t_in,
                          logL_restricted_M2 = d2t_in,
                          logL_unrestricted_M1 = d1f_in,
                          logL_unrestricted_M2 = d2f_in,
                          uhat_restricted_M1 = d1t_out,
                          uhat_restricted_M2 = d2t_out,
                          uhat_unrestricted_M1 = d1f_out,
                          uhat_unrestricted_M2 = d2f_out)
CSV.write("compare_daily.csv", compare_daily)

# this is a code chunk

logL_plot_daily, uhat_plot_daily = compare_plots(national_daily, compare_daily, "Daily");

# this is a code chunk

w1t_in, w1t_out = Out_Sample_Perform(national_weekly, 1, true, step);
w1f_in, w1f_out = Out_Sample_Perform(national_weekly, 1, false, step);
w2t_in, w2t_out = Out_Sample_Perform(national_weekly, 2, true, step);
w2f_in, w2f_out = Out_Sample_Perform(national_weekly, 2, false, step);

# this is a code chunk

compare_weekly = DataFrame(logL_restricted_M1 = w1t_in,
logL_restricted_M2 = w2t_in,
logL_unrestricted_M1 = w1f_in,
logL_unrestricted_M2 = w2f_in,
uhat_restricted_M1 = w1t_out,
uhat_restricted_M2 = w2t_out,
uhat_unrestricted_M1 = w1f_out,
uhat_unrestricted_M2 = w2f_out)
CSV.write("compare_weekly.csv", compare_weekly)

# this is a code chunk

logL_plot_weekly, uhat_plot_weekly = compare_plots(national_weekly, compare_weekly, "Weekly");

# this is a code chunk

dd71t_in, dd71t_out = Out_Sample_Perform(national_7dd, 1, true, step);
dd71f_in, dd71f_out = Out_Sample_Perform(national_7dd, 1, false, step);
dd72t_in, dd72t_out = Out_Sample_Perform(national_7dd, 2, true, step);
dd72f_in, dd72f_out = Out_Sample_Perform(national_7dd, 2, false, step);

# this is a code chunk

compare_7dd = DataFrame(logL_restricted_M1 = dd71t_in,
                          logL_restricted_M2 = dd72t_in,
                          logL_unrestricted_M1 = dd71f_in,
                          logL_unrestricted_M2 = dd72f_in,
                          uhat_restricted_M1 = dd71t_out,
                          uhat_restricted_M2 = dd72t_out,
                          uhat_unrestricted_M1 = dd71f_out,
                          uhat_unrestricted_M2 = dd72f_out)
CSV.write("compare_7dd.csv", compare_weekly) 

# this is a code chunk

logL_plot_7dd, uhat_plot_7dd = compare_plots(national_7dd, compare_7dd, "7DD");

# 1st Model
With locally constant assumption,
$$
\Delta c_t = \beta_t \Delta n_t + U_t
$$
The log-likelihood is:
$$
\ell(\beta_t) \propto - \frac{T}{2} \log(\sigma^2) - \frac{1}{2}\sum_{t=1}^T \frac{(y_t - \beta_t x_t)^2}{\sigma^2}
$$
The score is:
$$
\frac{\partial \ell}{\partial \beta_t} = 
\frac{1}{\sigma^2} (y_t - \beta_t x_t) x_t
$$

The hessian is:
$$
\frac{\partial^2 \ell}{\partial \beta_t^2} = 
- \frac{1}{\sigma^2} x_t^2
$$

Define
$$\psi(\beta_t) =
\frac{\frac{\partial \ell}{\partial \beta_t}}{\sqrt{- \frac{\partial^2 \ell}{\partial \beta_t^2}}}
=
\frac{\frac{1}{\sigma^2} (y_t - \beta_t x_t) x_t}{\sqrt{\frac{2}{\sigma^2} x_t^2}}
= 
\frac{1}{\sigma} \text{sign}(x_t) (y_t - \beta_t x_t)
$$

Score model:
$$\beta_{t+1} = \omega + \phi \beta_t + \alpha \psi(\beta_t)
=
\omega + \phi \beta_t +  \alpha \frac{\text{sign}(x_t) (y_t - \beta_t x_t)}{\sigma} 
$$

Re-parameterize: 
$$
\beta_t = \frac{1}{1 + \exp(-\theta_t)}
$$

The new log-likelihood is:

$$
\ell(\theta_t) \propto - \frac{T}{2} \log(\sigma^2) 
- 
\frac{1}{2 \sigma^2} 
\sum_{t=1}^T \left(y_t - \frac{x_t}{1 + \exp(-\theta_t)}\right)^2
;$$

The new score is:

$$
\frac{\partial \ell}{\partial \theta_t} 
= 
\frac{x_t}{\sigma^2  \exp(\theta_t) \left( 1 + \exp(-\theta_t) \right)^2}
\left(  y_t - \frac{x_t}{1 + \exp(-\theta_t)}  \right) 
$$

The new hessian is:
$$
\frac{\partial^2 \ell}{\partial \theta_t^2} 
=
- \frac{x_t^2}{\sigma^2   \exp(2 \theta_t)   (1 + \exp(-\theta_t))^4}
$$

The new $\psi$ is:
$$
\psi(\theta_t) =
\frac{\frac{\partial \ell}{\partial \theta_t}}{\sqrt{- \frac{\partial^2 \ell}{\partial \theta_t^2}}}
=
\frac{1}{\sigma} \text{sign}(x_t) \left(y_t -  \frac{x_t}{1 + \exp(-\theta_t)} \right)
$$

The new score model is:
$$
\theta_{t+1} = \omega + \phi \theta_t + \alpha \psi(\theta_t)
$$

# 2nd Model
Only ignore $\Delta \log(\pi^*_t)$, 
$$
\Delta c_t = 
\beta_t n_t - 
\beta_{t-1} n_{t-1} - 
\beta_t n^* +
\beta_{t-1} n^* +
U_t
$$

The log-likelihood is:
$$
\ell(\beta_t) \propto 
- \frac{T}{2}  \log(\sigma^2) 
- \frac{1}{2 \sigma^2} 
\sum_{t=1}^T  
(\Delta c_t - \beta_t n_t + 
\beta_{t-1} n_{t-1} +
\beta_t n^* -
\beta_{t-1} n^*)^2  
$$

The score is:
$$
\frac{\partial \ell}{\partial \beta_t} =
\frac{1}{\sigma^2}
(\Delta c_t - \beta_t n_t + 
\beta_{t-1} n_{t-1} +
\beta_t n^* -
\beta_{t-1} n^*)
(n_t - n^*)
$$

The hessian is:
$$
\frac{\partial^2 \ell}{\partial \beta_t^2} = 
- \frac{1}{\sigma^2}
(n^* - n_t)^2
$$


The $\psi(\beta_t)$ in the score model becomes:
$$
\psi(\beta_t) =
\frac{\text{sign}(n_t - n^*)}{\sigma}
(\Delta c_t - \beta_t n_t + 
\beta_{t-1} n_{t-1} +
\beta_t n^* -
\beta_{t-1} n^*)
$$

since $\text{sign}(n_t - n^*)$ is always negative, so $\psi(\beta_t)$ is,
$$
\psi(\beta_t) =
- \frac{1}{\sigma}
(\Delta c_t - \beta_t n_t + 
\beta_{t-1} n_{t-1} +
\beta_t n^* -
\beta_{t-1} n^*)
$$

Re-parameterize: 
$$
\beta_t = \frac{1}{1 + \exp(-\theta_t)}
$$

The new log-likelihood is:
$$
\begin{aligned}
\ell(\theta_t) \propto &
- \frac{T}{2}  \log(\sigma^2) \\ &
- \frac{1}{2 \sigma^2} 
\sum_{t=1}^T  
\left(
    \Delta c_t - \frac{n_t}{1 + e^{-\theta_t}}  + 
\frac{n_{t-1}}{1 + e^{-\theta_{t-1}}}  +
\frac{n^*}{1 + e^{-\theta_t}}  -
\frac{n^*}{1 + e^{-\theta_{t-1}}}
\right) ^2 
\end{aligned} 
$$

The new score is:
$$
\begin{aligned}
\frac{\partial \ell}{\partial \beta_t} \frac{\partial \beta_t}{\partial \theta_t} 
= &
\frac{n_t - n^*}{\sigma^2  e^{\theta_t} (1 + e^{-\theta_t})^2} 
\left(
    \Delta c_t - \frac{n_t}{1 + e^{-\theta_t}}  + 
\frac{n_{t-1}}{1 + e^{-\theta_{t-1}}}  +
\frac{n^*}{1 + e^{-\theta_t}}  -
\frac{n^*}{1 + e^{-\theta_{t-1}}} 
\right)
\end{aligned}
$$

Second derivative chain rule:
$$
\frac{\partial^2 \ell}{\partial \theta_t^2} =
\frac{\partial^2 \ell}{\partial \beta_t^2}  
\left(
\frac{\partial \beta_t}{\partial \theta_t}
\right)^2
+
\frac{\partial \ell}{\partial \beta_t} 
\frac{\partial^2 \beta_t}{\partial \theta_t^2}
$$

The new hessian is:
$$
\begin{aligned}
\frac{\partial^2 \ell}{\partial \theta_t^2} = &  
\mathbb{E} 
\left[
- \frac{1}{\sigma^2} (n^* - n_t)^2 
\frac{1}{e^{2 \theta_t} (1 + e^{-\theta_t})^4}
\right]
\\& +  
\mathbb{E}
\left[
\frac{1}{\sigma^2}
(\Delta c_t - \beta_t n_t + 
\beta_{t-1} n_{t-1} +
\beta_t n^* -
\beta_{t-1} n^*)
(n_t - n^*)  
\left(
\frac{2}{e^{2 \theta_t} (1 + e^{-\theta_t})^3}
-
\frac{1}{e^{\theta_t} (1 + e^{-\theta_t})^2}
\right)
\right] \\
= &
- \frac{1}{\sigma^2} (n^* - n_t)^2 
\frac{1}{e^{2 \theta_t} (1 + e^{-\theta_t})^4}
\end{aligned}
$$

The new $\psi(\theta_t)$ is:
$$
\psi(\theta_t) =
- \frac{1}{\sigma}
\left(
    \Delta c_t - \frac{n_t}{1 + e^{-\theta_t}}  + 
\frac{n_{t-1}}{1 + e^{-\theta_{t-1}}}  +
\frac{n^*}{1 + e^{-\theta_t}}  -
\frac{n^*}{1 + e^{-\theta_{t-1}}}
\right)
$$

The new score model is:
$$
\theta_{t+1} = \omega + \phi \theta_t + \alpha \psi(\theta_t)
$$

# 3rd Model
Combind model 1 and model 2. Rewrite:
$$
\Delta c_t = 
\beta_t n_t - 
\beta_{t-1} n_{t-1} - 
\beta_t n^* +
\beta_{t-1} n^* +
U_t
$$
as
$$
\Delta c_t = 
\beta_t \Delta n_t +
b_t \Delta \beta_t (n_{t-1} - n^*) +
U_t
$$
Then, if $b_t = 0$, model 1 is true, if $b_t = 1$, model 2 is true.
We can construct CI of $b_t$ to test the locally constant assumption.
The process of $b_t$ can be modeled by score as well. 
To be added...

# Results and Plots

## Daily

In [ ]:
plot(plot_ntct_daily, plot_ntct_daily,
     α_only1_daily_p2, α_only2_daily_p2,
     α_and_ϕ1_daily_p2, α_and_ϕ2_daily_p2,
layout = (3, 2),
left_margin = 50px,
right_margin = 50px,
xtickfont=(8), 
ytickfont=(8),
size = (1200, 1050))
#png("2-1-daily")


In [ ]:
plot(α_only1_daily_pUt, α_only2_daily_pUt,
     α_only1_daily_pUt2, α_only2_daily_pUt2,
     α_and_ϕ1_daily_pUt, α_and_ϕ2_daily_pUt,
     α_and_ϕ1_daily_pUt2, α_and_ϕ2_daily_pUt2,
layout = (4, 2),
left_margin = 50px,
right_margin = 50px,
xtickfont=(8), 
ytickfont=(8),
size = (1200, 1400))
#png("6-1-daily")

In [ ]:
plot(α_only1_daily_p1,α_only2_daily_p1,
     α_and_ϕ1_daily_p1, α_and_ϕ2_daily_p1,
    layout = (2, 2), size = (800, 800))
#png("daily1_1")

## Weekly

In [ ]:
plot(plot_ntct_weekly, plot_ntct_weekly,
     α_only1_weekly_p2, α_only2_weekly_p2,
     α_and_ϕ1_weekly_p2, α_and_ϕ2_weekly_p2,
layout = (3, 2),
left_margin = 50px,
right_margin = 50px,
xtickfont=(8), 
ytickfont=(8),
size = (1200, 1050))
#png("2-2-weekly")

In [ ]:
plot(α_only1_weekly_pUt, α_only2_weekly_pUt,
     α_only1_weekly_pUt2, α_only2_weekly_pUt2,
     α_and_ϕ1_weekly_pUt, α_and_ϕ2_weekly_pUt,
     α_and_ϕ1_weekly_pUt2, α_and_ϕ2_weekly_pUt2,
layout = (4, 2),
left_margin = 50px,
right_margin = 50px,
xtickfont=(8), 
ytickfont=(8),
size = (1200, 1400))
#png("6-2-weekly")

In [ ]:
plot(α_only1_weekly_p1, α_only2_weekly_p1, 
     α_and_ϕ1_weekly_p1, α_and_ϕ2_weekly_p1,
    layout = (2, 2), size = (800, 800))
#png("weekly1_1")

## 7-Day-Difference

In [ ]:
plot(plot_ntct_daily, plot_ntct_daily,
     α_only1_7dd_p2, α_only2_7dd_p2,
     α_and_ϕ1_7dd_p2, α_and_ϕ2_7dd_p2,
layout = (3, 2),
left_margin = 50px,
right_margin = 50px,
xtickfont=(8), 
ytickfont=(8),
size = (1200, 1050))
#png("2-3-7dd")

In [ ]:
plot(α_only1_7dd_pUt, α_only2_7dd_pUt,
     α_only1_7dd_pUt2, α_only2_7dd_pUt2,
     α_and_ϕ1_7dd_pUt, α_and_ϕ2_7dd_pUt,
     α_and_ϕ1_7dd_pUt2, α_and_ϕ2_7dd_pUt2,
layout = (4, 2),
left_margin = 50px,
right_margin = 50px,
xtickfont=(8), 
ytickfont=(8),
size = (1200, 1400))
#png("6-3-7dd")

In [ ]:
plot(α_only1_7dd_p1, α_only2_7dd_p1, 
     α_and_ϕ1_7dd_p1, α_and_ϕ2_7dd_p1,
    layout = (2, 2), size = (800, 800))
#png("7dd1_1")

## Out-of-Sample Compare

# this is a code chunk

plot(logL_plot_daily,logL_plot_weekly,logL_plot_7dd, 
     uhat_plot_daily, uhat_plot_weekly, uhat_plot_7dd,
     layout = (2, 3),
     size = (1200, 600),
     leg = :topleft)
#png("4-Out-Sample")